hihello 예제는 입력이 hihello일때만 동작한다.</br>
여기서 다루는 예제는 어떤 문자들이 들어와도 동작하는 일반화된 모델을 만든다.</br>

In [2]:
import torch
import torch.optim as optim
import numpy as np

torch.manual_seed(0)

sample = " if you want you"

# 중복된 문자를 제거하고 unique한 문자들의 리스트를 만든다.
char_set = list(set(sample))
# 인덱스와 문자를 매핑한 dictionary를 만든다.
char_dic = {c: i for i, c in enumerate(char_set)}

# unique한 문자의 개수를 input_size로 한다.
input_size = len(char_set)
# 다른 값으로 설정해도 되지만 여기서는 input_size와 동일하게 설정한다.
hidden_size = len(char_dic)
learning_rate = 0.1

# 입력 데이터의 문자를 인덱스로 변환한다.
sample_idx = [char_dic[c] for c in sample]
# 문자열의 맨마지막 문자만 제외한다.
x_data = [sample_idx[:-1]]
# np.eye 함수는 identity 행렬을 만들어준다.
# identity 행렬을 활용해서 one-hot 인코딩을 한다.
x_one_hot = [np.eye(input_size)[x] for x in x_data]
# 문자열의 맨처음 문자만 제외한다.
y_data = [sample_idx[1:]]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

# start training
for i in range(50):
    optimizer.zero_grad()       # gradient를 초기화한다.
    # _status는 hidden state인데, 여기서는 한번의 입력으로 끝내기 때문에 사용되지 않는다.
    outputs, _status = rnn(X)
    # view(-1)은 batch demension을 맨앞으로 오게 한다.
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()     # back propagation을 진행한다. 
    optimizer.step()    # 최적화를 통해서 파라미터를 업데이트한다.

    # argmax 함수는 값이 가장 큰 인덱스만 가져온다. 
    # 3번째 차원의 값들중 가장 큰 값의 인덱스를 가져온다. 
    result = outputs.data.numpy().argmax(axis=2)
    # result의 인덱스가 어떤 문자에 해당하는지 가져와서 하나의 문자열로 합친다.
    # squeeze 함수는 shape에서 demension이 1인 축을 없애준다.
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)


0 loss:  2.388148546218872 prediction:  [[8 7 8 5 7 7 0 5 7 6 7 7 8 7 7]] true Y:  [[0, 8, 1, 4, 3, 7, 1, 2, 9, 5, 6, 1, 4, 3, 7]] prediction str:  fufnuuinutuufuu
1 loss:  2.0313820838928223 prediction:  [[8 1 1 1 8 7 1 0 8 7 1 1 1 7 7]] true Y:  [[0, 8, 1, 4, 3, 7, 1, 2, 9, 5, 6, 1, 4, 3, 7]] prediction str:  f   fu ifu   uu
2 loss:  1.8183174133300781 prediction:  [[4 1 1 4 8 7 1 2 4 5 1 1 4 8 7]] true Y:  [[0, 8, 1, 4, 3, 7, 1, 2, 9, 5, 6, 1, 4, 3, 7]] prediction str:  y  yfu wyn  yfu
3 loss:  1.6600782871246338 prediction:  [[4 1 1 4 3 7 1 6 4 5 1 1 4 3 7]] true Y:  [[0, 8, 1, 4, 3, 7, 1, 2, 9, 5, 6, 1, 4, 3, 7]] prediction str:  y  you tyn  you
4 loss:  1.5549092292785645 prediction:  [[4 3 1 4 3 7 1 6 3 5 6 1 4 3 7]] true Y:  [[0, 8, 1, 4, 3, 7, 1, 2, 9, 5, 6, 1, 4, 3, 7]] prediction str:  yo you tont you
5 loss:  1.499359130859375 prediction:  [[4 3 1 4 3 7 1 4 9 5 6 4 4 3 7]] true Y:  [[0, 8, 1, 4, 3, 7, 1, 2, 9, 5, 6, 1, 4, 3, 7]] prediction str:  yo you yantyyou
6 loss:  1.4